# PRO141 – HMI interactivo

Incluye selector inicial de alcance (**Análisis de obsolescencia** vs **Tratamiento de materiales obsoletos**).

In [8]:
import ipywidgets as widgets
from IPython.display import display
import datetime, json, uuid

# ============================================================
# PRO141 – Tratamiento de materiales obsoletos y análisis de obsolescencia
# HMI oficial (BASE CLON PRO134 aprobado)
#
# Selector inicial obligatorio:
#   - Análisis de obsolescencia
#   - Tratamiento de materiales obsoletos
#
# Reglas HMI:
# - Títulos de pasos = casillas del flujo (texto exacto)  [excepto selectores HMI]
# - Checklist por paso (no avanza si no se completa)
# - Botón NO => BLOQUEADO con motivo obligatorio + "Rehacer paso"
# - Volver al paso anterior
# - Exporta JSON auditable (run_id, historial, decisiones, bloqueos, timestamps, estados)
# ============================================================

EN_CURSO = "EN_CURSO"
BLOQUEADO = "BLOQUEADO"
DETENIDO_STOP = "DETENIDO_STOP"   # (no usado en este PRO, se mantiene por estándar)
FINALIZADO = "FINALIZADO"

def _now_iso():
    return datetime.datetime.now().isoformat(timespec="seconds")

# Motivos de bloqueo (multi-selección) – PRO141
MOTIVOS_BLOQUEO_PRO141 = [
    "Falta información mínima para formulario / reporte",
    "Aprobación rechazada / falta firma",
    "No se puede ejecutar transacción SAP",
    "Material no segregado / riesgo de mezcla",
    "Discrepancia stock físico vs sistema",
    "Falta respuesta de otra central",
    "Otro"
]

# ------------------------------------------------------------
# NODOS – PRO141 (Análisis + Tratamiento)
# ------------------------------------------------------------
NODOS = {
    # ===== Selector de alcance (HMI) =====
    "S0_alcance": {
        "type": "decision",
        "titulo": "¿Se está realizando un análisis de obsolescencia o tratamiento de materiales obsoletos?",
        "rol": "HMI (selección de alcance)",
        "descripcion": "Seleccione el flujo correcto según el objetivo del trabajo.",
        "pregunta": "Seleccione una opción para iniciar:",
        "opciones": [
            {"label": "Análisis de obsolescencia", "next": "AN1_inicio"},
            {"label": "Tratamiento de materiales obsoletos", "next": "TR0_inicio"}
        ],
        "ayuda": "Este selector controla el alcance del procedimiento antes de ejecutar acciones."
    },

    # ============================================================
    # FLUJO A: ANÁLISIS DE OBSOLESCENCIA (según diagrama PRO141)
    # ============================================================
    "AN1_inicio": {
        "type": "task",
        "titulo": "Inicio de análisis de obsolescencia",
        "rol": "Jefe de ingeniería de materiales/ Jefe de Planificación y Predictivo (GSO)",
        "descripcion": "Inicio del proceso de provisión/analítica anual de obsolescencia.",
        "acciones": [
            "Iniciar el análisis anual según calendario definido (referencia: mes de Septiembre)."
        ],
        "checklist": [
            "Inicio registrado",
            "Período de análisis definido"
        ],
        "validacion": "¿Se inició el análisis de obsolescencia y quedó definido el período?",
        "next": "AN2_crear_reporte"
    },
    "AN2_crear_reporte": {
        "type": "task",
        "titulo": "Crear reporte de Universo de materiales y repuestos",
        "rol": "Jefe de ingeniería de materiales/ Jefe de Planificación y Predictivo (GSO)",
        "descripcion": "Preparar el reporte de materiales y repuestos en desuso bajo los criterios del procedimiento.",
        "acciones": [
            "Preparar el reporte del universo de materiales y repuestos en desuso, considerando criterios del procedimiento:",
            "1) Materiales y repuestos asociados a Químicas o Instrumentación y Control, con tiempo de desuso >= 3 años.",
            "2) Repuestos de equipos con estrategia a la falla, que han fallado en los últimos 5 años y no presentan consumo en los últimos 5 años.",
            "3) Repuestos de equipos con estrategia de mantenimiento preventiva que no han registrado consumo durante 3 o más ciclos de mantenimiento, aunque el plan plantee uso periódico del repuesto.",
            "4) Repuestos con estrategia de mantenimiento preventiva (en almacén) que no ha registrado OT en 3 o más ciclos de mantenimiento del repuesto.",
            "5) Repuestos que no representan vínculo a un equipo o ubicación técnica."
        ],
        "checklist": [
            "Universo de materiales/repuestos generado",
            "Criterios aplicados",
            "Listado trazable (códigos/cantidades)"
        ],
        "validacion": "¿El reporte del universo de materiales y repuestos fue creado y es trazable?",
        "next": "AN3_enviar_reporte"
    },
    "AN3_enviar_reporte": {
        "type": "task",
        "titulo": "Enviar reporte",
        "rol": "Jefe de ingeniería de materiales/ Jefe de Planificación y Predictivo (GSO)",
        "descripcion": "Enviar el reporte para habilitar la provisión y pasos posteriores al subgerente de contabilidad.",
        "acciones": [
            "Enviar el reporte del universo de materiales y repuestos según distribución definida por el procedimiento."
        ],
        "checklist": [
            "Reporte enviado",
            "Destino(s) confirmados",
            "Registro de envío disponible"
        ],
        "validacion": "¿El reporte fue enviado y quedó registro del envío?",
        "next": "AN4_registrar_provision"
    },
    "AN4_registrar_provision": {
        "type": "task",
        "titulo": "Registrar provisión de obsolescencia",
        "rol": "Subgerente de contabilidad",
        "descripcion": "Registrar provisión anual de obsolescencia según reporte recibido.",
        "acciones": [
            "Registrar provisión de obsolescencia en base al reporte recibido."
        ],
        "checklist": [
            "Provisión registrada",
            "Registro/soporte disponible"
        ],
        "validacion": "¿La provisión de obsolescencia fue registrada y es trazable?",
        "next": "AN5_recepcion_listado"
    },
    "AN5_recepcion_listado": {
        "type": "task",
        "titulo": "Recepción de listado de materiales",
        "rol": "Gerente de Tecnología",
        "descripcion": "Recepción del listado asociado al análisis/provisión para coordinar revisión y retroalimentación.",
        "acciones": [
            "Recibir listado de materiales y repuestos resultantes del análisis/provisión."
        ],
        "checklist": [
            "Listado recibido",
            "Listado disponible para coordinación"
        ],
        "validacion": "¿El listado fue recibido y está disponible para coordinar revisión?",
        "next": "AN6_coordinar_revision"
    },
    "AN6_coordinar_revision": {
        "type": "task",
        "titulo": "Coordinar revisión y retroalimentación de los resultados del análisis de obsolescencia",
        "rol": "Gerente de Tecnología",
        "descripcion": "Coordinar revisión y retroalimentación de resultados dentro de los plazos del procedimiento.",
        "acciones": [
            "Coordinar revisión y retroalimentación de los resultados del análisis de obsolescencia.",
            "Asegurar retroalimentación dentro del plazo establecido por el procedimiento (referencia: último plazo en Agosto del año siguiente)."
        ],
        "checklist": [
            "Revisión coordinada",
            "Retroalimentación registrada"
        ],
        "validacion": "¿La revisión y retroalimentación de resultados fue coordinada y registrada?",
        "next": "AN7_tratamiento_subproceso"
    },
    "AN7_tratamiento_subproceso": {
        "type": "task",
        "titulo": "Tratamiento de materiales obsoletos",
        "rol": "Gerente de Tecnología",
        "descripcion": "Subproceso: derivación al flujo de tratamiento de materiales obsoletos (cuando aplique).",
        "acciones": [
            "Derivar al flujo de tratamiento de materiales obsoletos según el listado definido."
        ],
        "checklist": [
            "Derivación realizada",
            "Listado/alcance definido para tratamiento"
        ],
        "validacion": "¿Se derivó correctamente al tratamiento de materiales obsoletos (si aplica)?",
        "next": "TR0_inicio"
    },

    # ============================================================
    # FLUJO B: TRATAMIENTO DE MATERIALES OBSOLETOS (Flujo 1)
    # ============================================================
    "TR0_inicio": {
        "type": "decision",
        "titulo": "¿Cómo se inicia el tratamiento de material obsoleto?",
        "rol": "HMI (selección de inicio)",
        "descripcion": "Seleccione el punto de entrada según el contexto en terreno.",
        "pregunta": "Seleccione una opción para iniciar:",
        "opciones": [
            {"label": "Identificación de repuesto obsoleto por usuario central", "next": "TR_A1_identificacion"},
            {"label": "Necesidad de análisis individual (almacén)", "next": "TR_B1_solicitar_inspeccion"},
            {"label": "Recepción de análisis masivo", "next": "TR_C1_recepcion_masiva"}
        ],
        "ayuda": "Este selector define el punto de entrada sin modificar el flujo del procedimiento."
    },

    # ---- Inicio Usuario central ----
    "TR_A1_identificacion": {
        "type": "task",
        "titulo": "Identificación de repuesto obsoleto",
        "rol": "Usuario central",
        "descripcion": "Se identifica material o repuesto obsoleto.",
        "acciones": ["Identificar material/repuesto y notificar a almacenamiento."],
        "checklist": ["Material/repuesto identificado", "Notificación preparada"],
        "validacion": "¿El repuesto obsoleto fue identificado y está listo para informar?",
        "next": "TR_A2_informar_almacen"
    },
    "TR_A2_informar_almacen": {
        "type": "task",
        "titulo": "Informar a almacenamiento",
        "rol": "Usuario central",
        "descripcion": "Se informa a almacenamiento la identificación del repuesto obsoleto.",
        "acciones": ["Informar a almacenamiento por canal definido."],
        "checklist": ["Almacén informado", "Registro del aviso disponible"],
        "validacion": "¿Se informó a almacenamiento y quedó registro?",
        "next": "TR_F1_formulario"
    },

    # ---- Inicio Especialista de almacenamiento ----
    "TR_B1_solicitar_inspeccion": {
        "type": "task",
        "titulo": "Solicitar inspección de materiales",
        "rol": "Especialista de almacenamiento",
        "descripcion": "Solicitud de inspección de materiales por necesidad de análisis individual.",
        "acciones": ["Solicitar inspección de materiales.", "Segregar material si corresponde."],
        "checklist": ["Inspección solicitada", "Material segregado (si aplica)"],
        "validacion": "¿La inspección fue solicitada y el material quedó bajo control?",
        "next": "TR_B2_definir_inspector"
    },

    # ---- Inicio Encargado de materiales ----
    "TR_C1_recepcion_masiva": {
        "type": "task",
        "titulo": "Recepción de análisis masivo",
        "rol": "Encargado de materiales",
        "descripcion": "Recepción del análisis masivo para definir inspección.",
        "acciones": ["Recibir análisis masivo y preparar definición de inspección."],
        "checklist": ["Análisis masivo recibido"],
        "validacion": "¿Se recibió el análisis masivo?",
        "next": "TR_C2_definir_inspeccion"
    },
    "TR_C2_definir_inspeccion": {
        "type": "task",
        "titulo": "Definir inspección de materiales",
        "rol": "Encargado de materiales",
        "descripcion": "Definir materiales que requieren inspección.",
        "acciones": ["Definir inspección de materiales."],
        "checklist": ["Inspección definida"],
        "validacion": "¿La inspección de materiales quedó definida?",
        "next": "TR_B2_definir_inspector"
    },

    # ---- Tramo común: inspector / formulario / aprobación ----
    "TR_B2_definir_inspector": {
        "type": "task",
        "titulo": "Definir inspector de materiales",
        "rol": "Responsable de mtto./ Supervisor SSO/Jefe de administración",
        "descripcion": "Asignación de inspector de materiales.",
        "acciones": ["Definir inspector de materiales."],
        "checklist": ["Inspector definido"],
        "validacion": "¿El inspector de materiales fue definido?",
        "next": "TR_I1_analizar_obsolescencia"
    },
    "TR_I1_analizar_obsolescencia": {
        "type": "task",
        "titulo": "Analizar obsolescencia, condición y/o vencimiento del material",
        "rol": "Inspector de materiales",
        "descripcion": "Evaluación del material para determinar obsolescencia, condición o vencimiento.",
        "acciones": ["Analizar obsolescencia, condición y/o vencimiento del material."],
        "checklist": ["Análisis realizado", "Resultado definido"],
        "validacion": "¿El material fue analizado y se definió resultado?",
        "next": "TR_F1_formulario"
    },
    "TR_F1_formulario": {
        "type": "task",
        "titulo": "Llenar formulario de revisión de obsolescencia",
        "rol": "Especialista de almacenamiento",
        "descripcion": "Completar formulario de revisión de obsolescencia.",
        "acciones": ["Llenar formulario de revisión de obsolescencia."],
        "checklist": ["Formulario completado"],
        "validacion": "¿El formulario fue completado correctamente?",
        "next": "TR_F2_solicitar_aprobacion"
    },
    "TR_F2_solicitar_aprobacion": {
        "type": "task",
        "titulo": "Solicitar aprobación",
        "rol": "Especialista de almacenamiento",
        "descripcion": "Solicitar aprobación del formulario de revisión de obsolescencia.",
        "acciones": ["Solicitar aprobación."],
        "checklist": ["Aprobación solicitada"],
        "validacion": "¿Se solicitó la aprobación?",
        "next": "TR_F3_revisar_firmar"
    },
    "TR_F3_revisar_firmar": {
        "type": "task",
        "titulo": "Revisar y firmar formulario de obsolescencia",
        "rol": "Responsable de mtto./ Supervisor SSO/Jefe de administración",
        "descripcion": "Revisión y firma del formulario de obsolescencia.",
        "acciones": ["Revisar y firmar formulario de obsolescencia."],
        "checklist": ["Formulario revisado"],
        "validacion": "¿El formulario fue revisado y firmado?",
        "next": "TR_F4_firmar_formulario"
    },
    "TR_F4_firmar_formulario": {
        "type": "task",
        "titulo": "Firmar formulario de obsolescencia",
        "rol": "Especialista de almacenamiento",
        "descripcion": "Firma del formulario de obsolescencia según flujo.",
        "acciones": ["Firmar formulario de obsolescencia."],
        "checklist": ["Formulario firmado por especialista de almacenamiento"],
        "validacion": "¿El formulario fue firmado por especialista de almacenamiento?",
        "next": "TR_L1_traspasar_AL00"
    },

    # ---- AL00 / etiquetado / traslado ----
    "TR_L1_traspasar_AL00": {
        "type": "task",
        "titulo": "Traspasar material a AL00",
        "rol": "Encargado de materiales",
        "descripcion": "Movimiento 311 – Traspasar material a AL00.",
        "acciones": ["Traspasar material a AL00 (movimiento 311)."],
        "checklist": ["Movimiento ejecutado", "Documento registrado"],
        "validacion": "¿El material fue traspasado a AL00 en sistema?",
        "next": "TR_L2_modificar_ubicacion"
    },
    "TR_L2_modificar_ubicacion": {
        "type": "task",
        "titulo": "Modificar ubicación del material",
        "rol": "Encargado de materiales",
        "descripcion": "MM02 – Modificar ubicación del material.",
        "acciones": ["Modificar ubicación del material (MM02)."],
        "checklist": ["Ubicación modificada", "Físico y sistema coinciden"],
        "validacion": "¿La ubicación del material fue modificada y coincide con físico?",
        "next": "TR_L3_etiquetar"
    },
    "TR_L3_etiquetar": {
        "type": "task",
        "titulo": "Etiquetar material",
        "rol": "Encargado de materiales",
        "descripcion": "ZMM_IMP_ETIQUETA – Imprimir etiqueta e identificar material.",
        "acciones": ["Imprimir etiqueta y etiquetar material (ZMM_IMP_ETIQUETA)."],
        "inputs": [
            {
                "key": "numero_etiqueta",
                "label": "Introduce número de etiqueta",
                "required": True,
                "placeholder": "Ej: ETQ-12345"
            }
        ],
        "checklist": ["Etiqueta impresa", "Material etiquetado"],
        "validacion": "¿El material quedó etiquetado?",
        "next": "TR_L4_trasladar"
    },
    "TR_L4_trasladar": {
        "type": "task",
        "titulo": "Trasladar material obsoleto",
        "rol": "Encargado de materiales",
        "descripcion": "Traslado físico del material obsoleto.",
        "acciones": ["Trasladar material obsoleto a ubicación definida (AL00)."],
        "checklist": ["Material trasladado", "Material segregado"],
        "validacion": "¿El material fue trasladado y quedó segregado en AL00?",
        "next": "TR_L5_analizar_utilidad_otras"
    },

    # ---- Otras centrales: verificación + decisión ----
    "TR_L5_analizar_utilidad_otras": {
        "type": "task",
        "titulo": "Analizar utilidad material en otras centrales",
        "rol": "Encargado de materiales",
        "descripcion": "Analizar utilidad del material en otras centrales.",
        "acciones": ["Enviar consulta a otras centrales para evaluar utilidad del material."],
        "checklist": ["Consulta enviada", "Datos suficientes enviados"],
        "validacion": "¿Se consultó a otras centrales por la utilidad del material?",
        "next": "TR_L6_informar_utilidad"
    },
    "TR_L6_informar_utilidad": {
        "type": "task",
        "titulo": "Informar utilidad del material",
        "rol": "Especialista de Almacén de otra central",
        "descripcion": "Especialista de Almacén de otra central informa si el material es útil.",
        "acciones": ["Verificar con mtto/operación y responder utilidad del material."],
        "checklist": ["Utilidad verificada", "Respuesta enviada"],
        "validacion": "¿La otra central verificó e informó la utilidad del material?",
        "next": "TR_D1_util"
    },
    "TR_D1_util": {
        "type": "decision",
        "titulo": "¿El material es útil en otra central?",
        "rol": "Encargado de materiales",
        "descripcion": "Decisión según respuesta de otra central.",
        "pregunta": "¿El material es útil en otra central?",
        "opciones": [
            {"label": "SÍ", "next": "TR_D3_TRASLADO_A_OTRA_PLANTA"},
            {"label": "NO", "next": "TR_L7_inicio_desguace"}
        ]
    },

    # ---- Inicio de proceso de desguace (cuando se necesite: mínimo cada 6 meses) ----
    "TR_L7_inicio_desguace": {
        "type": "task",
        "titulo": "Inicio de proceso de desguace (cuando se necesite - mínimo cada 6 meses)",
        "rol": "Encargado de materiales",
        "descripcion": "Inicio del proceso de desguace según necesidad y frecuencia mínima definida.",
        "acciones": ["Iniciar proceso de desguace cuando se necesite (mínimo cada 6 meses)."],
        "checklist": ["Proceso de desguace iniciado"],
        "validacion": "¿Se inició el proceso de desguace según el criterio del procedimiento?",
        "next": "TR_L8_comercial"
    },

    # ---- Comercialización + decisión ----
    "TR_L8_comercial": {
        "type": "task",
        "titulo": "Analizar posibilidad de comercializar materiales en AL00",
        "rol": "Encargado de materiales",
        "descripcion": "Analizar posibilidad de comercializar materiales en AL00.",
        "acciones": ["Analizar posibilidad de comercializar materiales en AL00."],
        "checklist": ["Análisis de comercialización realizado"],
        "validacion": "¿Se analizó la posibilidad de comercializar materiales en AL00?",
        "next": "TR_D2_comercial"
    },
    "TR_D2_comercial": {
        "type": "decision",
        "titulo": "¿El material es comercializable?",
        "rol": "Encargado de materiales",
        "descripcion": "Decisión de salida final del material.",
        "pregunta": "¿El material es comercializable?",
        "opciones": [
            {"label": "SÍ", "next": "END_VENTAS"},
            {"label": "NO", "next": "END_DISPOSICION"}
        ]
    },
    "TR_D3_TRASLADO_A_OTRA_PLANTA": {
    "type": "task",
    "titulo": "Traslado del material a otra planta",
    "rol": "Bodega / Logística",
    "descripcion": "Registrar destino del material antes del cierre del proceso.",
    "acciones": [
        "Definir a qué planta/central se trasladará el material."
    ],
    "inputs": [
        {"key": "planta_destino", "label": "¿A qué planta/central irá el material?", "required": True}
    ],
    "checklist": [
        "Destino de traslado definido"
    ],
    "validacion": "¿Se registró el destino del traslado?",
    "next": "END_TRASLADO"
},

    # ---- Finales (con descripción específica) ----
    "END_TRASLADO": {
        "type": "end",
        "titulo": "🏁 Proceso finalizado",
        "rol": "HMI",
        "descripcion": "Resultado: traslado del material a otra central.",
        "mensaje": "Se coordina y ejecuta el traslado del material entre centrales según definición del procedimiento.",
        "inputs": [
            {
                "key": "centro_destino",
                "label": "¿A qué centro se fue trasladado?",
                "required": True,
                "placeholder": "Ej: Central X / AL00 / Bodega ..."
            }
        ],

        "estado_final": FINALIZADO
    },
    "END_VENTAS": {
        "type": "end",
        "titulo": "🏁 Proceso finalizado",
        "rol": "HMI",
        "descripcion": "Resultado: gestión de ventas varias.",
        "mensaje": "Se gestiona el material como ventas varias según definición del procedimiento.",
        "estado_final": FINALIZADO
    },
    "END_DISPOSICION": {
        "type": "end",
        "titulo": "🏁 Proceso finalizado",
        "rol": "HMI",
        "descripcion": "Resultado: disposición final del material.",
        "mensaje": "Se gestiona la disposición final del material según definición del procedimiento.",
        "estado_final": FINALIZADO
    }
}

# -------------------------
# HMI (CLON EXACTO motor PRO134)
# -------------------------
class PRO141HMI:
    def __init__(self):
        self.nodo_id = "S0_alcance"
        self.historial = []
        self.logs = []
        self.decisiones = []
        self.bloqueos = []
        self.inputs = {} # Initialize self.inputs

        self.run_id = str(uuid.uuid4())
        self.estado = EN_CURSO
        self.start_ts = _now_iso()
        self.end_ts = None

        self.output = widgets.Output(layout={"width":"100%"})

        self.btn_si = widgets.Button(description="SÍ", button_style="success", layout={"width":"48%","height":"44px"})
        self.btn_no = widgets.Button(description="NO", button_style="danger", layout={"width":"48%","height":"44px"})
        self.btn_volver = widgets.Button(description="⬅ Volver al paso anterior", layout={"width":"100%","height":"40px"})
        self.btn_exportar = widgets.Button(description="Exportar JSON (trazabilidad)", icon="download", layout={"width":"100%","height":"40px"})

        self.msg_box = widgets.HTML("")
        self.main_box = widgets.VBox([])

        self.is_blocked = False
        self.block_panel = widgets.VBox([])
        self.btn_rehacer = widgets.Button(description="🔄 Rehacer paso", button_style="info", layout={"width":"100%","height":"40px"})
        self.btn_rehacer.on_click(self._on_rehacer)

        self._decision_widget = None
        self._check_widgets = []

        self._wire()
        self._render()

    def _wire(self):
        self.btn_si.on_click(self._on_si)
        self.btn_no.on_click(self._on_no)
        self.btn_volver.on_click(self._on_volver)
        self.btn_exportar.on_click(self._on_exportar)

    def _log(self, tipo, data=None):
        self.logs.append({
            "ts": _now_iso(),
            "tipo": tipo,
            "estado": self.estado,
            "nodo": self.nodo_id,
            "data": data or {}
        })

    def _push_hist(self):
        self.historial.append(self.nodo_id)

    def _pop_hist(self):
        if self.historial:
            return self.historial.pop()
        return None

    def _set_msg(self, html):
        self.msg_box.value = html

    def _clear_msg(self):
        self.msg_box.value = ""

    def _render_header(self, n):
        badge = f"<span style='display:inline-block;padding:4px 10px;border-radius:999px;background:#eef2ff;border:1px solid #c7d2fe;font-size:12px;color:black;'><b>ROL:</b> {n.get('rol','')}</span>"
        return widgets.HTML(f"""
        <div style="padding:16px;border-radius:12px;background:#f8fafc;border:1px solid #e2e8f0;">
            <div style="font-size:12px;color:#0f172a;"><b>PRO141</b> – Tratamiento de materiales obsoletos y análisis de obsolescencia</div>
            <div style="margin-top:6px;font-size:22px;color:#0f172a;"><b>{n.get('titulo','')}</b></div>
            <div style="margin-top:8px;">{badge}</div>
            <div style="margin-top:10px;color:#0f172a;font-size:14px;line-height:1.35;">{n.get('descripcion','')}
        </div>
        """)

    def _render_task(self, n):
        acciones = "".join([f"<li style='margin:4px 0;color:#0f172a;'>{a}</li>" for a in n.get("acciones",[])])
        valid = n.get("validacion","")

        self._check_widgets = [widgets.Checkbox(description=item, value=False) for item in (n.get("checklist",[]) or [])]

        # ---- Inputs (guardados en JSON) ----
        self._input_widgets = []
        input_specs = n.get("inputs", []) or []
        inputs_box = widgets.VBox([])
        if input_specs:
            rows = []
            for spec in input_specs:
                key = spec.get("key")
                label = spec.get("label", key)
                required = bool(spec.get("required", False))
                placeholder = spec.get("placeholder", "")
                w = widgets.Text(
                    value=str(self.inputs.get(key, "")) if key else "",
                    placeholder=placeholder,
                    layout=widgets.Layout(width="100%")
                )
                # metadata para validación/guardado
                w._pro_key = key
                w._pro_required = required
                w._pro_label = label
                self._input_widgets.append(w)

                rows.append(widgets.VBox([
                    widgets.HTML(f"<div style='font-size:12px;color:#0f172a;'><b>{label}{' *' if required else ''}</b></div>"),
                    w
                ]))

            inputs_box = widgets.VBox([
                widgets.HTML("""
                <div style="margin-top:12px;padding:14px;border-radius:12px;border:1px solid #e2e8f0;background:#ffffff;">
                    <div style="font-size:13px;color:#0f172a;"><b>📝 REGISTRO (Inputs)</b></div>
                    <div style="margin-top:8px;font-size:12px;color:#0f172a;opacity:0.9;">Campos con * son obligatorios.</div>
                </div>
                """),
                widgets.VBox(rows)
            ])

        accion_box = widgets.HTML(f"""
            <div style="margin-top:12px;padding:14px;border-radius:12px;border:1px solid #e2e8f0;background:#ffffff;">
                <div style="font-size:13px;color:#0f172a;"><b>⚙️ ACCIÓN A EJECUTAR (texto PRO141)</b></div>
                <ul style="margin-top:10px;padding-left:18px;color:#0f172a;">{acciones}</ul>
            </div>
        """)

        checklist_box = widgets.VBox([])
        if self._check_widgets:
            checklist_box = widgets.VBox([
                widgets.HTML("""
                <div style="margin-top:12px;padding:14px;border-radius:12px;border:1px solid #e2e8f0;background:#ffffff;">
                    <div style="font-size:13px;color:#0f172a;"><b>🧾 CHECKLIST (obligatorio para avanzar)</b></div>
                    <div style="margin-top:8px;font-size:12px;color:#0f172a;opacity:0.9;">Marca cada ítem al completar.</div>
                </div>
                """),
                widgets.VBox(self._check_widgets)
            ])

        valid_box = widgets.HTML(f"""
            <div style="margin-top:12px;padding:14px;border-radius:12px;border:2px solid #0ea5e9;background:#ffffff;">
                <div style="font-size:13px;color:#0f172a;"><b>✅ ¡VALIDACIÓN!</b></div>
                <div style="margin-top:8px;font-size:16px;color:#0f172a;"><b>{valid}</b></div>
                <div style="margin-top:6px;font-size:12px;color:#0f172a;">Confirma con <b>SÍ</b> para avanzar. Si respondes <b>NO</b>, el paso queda bloqueado.</div>
            </div>
        """)

        return widgets.VBox([accion_box, inputs_box, checklist_box, valid_box])

    def _render_decision(self, n):
        opts = n.get("opciones",[])
        radios = widgets.RadioButtons(
            options=[(o["label"], o["next"]) for o in opts],
            layout={"width":"100%"},
            style={"description_width":"initial"}
        )
        help_txt = n.get("ayuda","")
        help_html = f"<div style='margin-top:10px;font-size:12px;color:#0f172a;opacity:0.9;'><b>Nota:</b> {help_txt}</div>" if help_txt else ""
        return widgets.VBox([
            widgets.HTML(f"""
            <div style="margin-top:12px;padding:14px;border-radius:12px;border:1px solid #e2e8f0;background:#ffffff;">
                <div style="font-size:13px;color:#0f172a;"><b>🔶 DECISIÓN (rombo)</b></div>
                <div style="margin-top:8px;font-size:16px;color:#0f172a;"><b>{n.get('pregunta','')}</b></div>
                {help_html}
            </div>
            """),
            radios
        ]), radios

    def _render_block_panel(self):
        if not self.is_blocked:
            self.block_panel.children = []
            return

        title = widgets.HTML("""
        <div style="margin-top:12px;padding:14px;border-radius:12px;border:2px solid #ef4444;background:#fff1f2;">
            <div style="font-size:14px;color:#0f172a;"><b>⛔ BLOQUEADO</b> — Seleccione motivo(s) y registre detalle.</div>
            <div style="margin-top:8px;font-size:12px;color:#0f172a;">No puede avanzar hasta rehacer el paso.</div>
        </div>
        """)

        self.sel_motivos = widgets.SelectMultiple(options=MOTIVOS_BLOQUEO_PRO141, rows=7, layout={"width":"100%"})
        self.txt_detalle = widgets.Textarea(
            placeholder="Detalle del bloqueo (obligatorio si selecciona 'Otro').",
            layout=widgets.Layout(width="100%", height="80px")
        )

        self.block_panel.children = [
            title,
            widgets.HTML("<b>Motivo(s) de bloqueo:</b> (selección múltiple)"),
            self.sel_motivos,
            widgets.HTML("<b>Detalle:</b>"),
            self.txt_detalle,
            self.btn_rehacer
        ]


    def _render_tables_if_end(self, n):
        """Tablas compactas: Inputs + Decisiones (solo nodos de decisión)."""
        if n.get("type") != "end":
            return widgets.VBox([])

        # Inputs
        inputs_rows = ""
        for k, v in (self.inputs or {}).items():
            inputs_rows += f"<tr><td style='border:1px solid #e2e8f0;padding:4px;'><b>{k}</b></td><td style='border:1px solid #e2e8f0;padding:4px;'>{(v or '')}</td></tr>"
        if not inputs_rows:
            inputs_rows = "<tr><td colspan='2' style='border:1px solid #e2e8f0;padding:4px;'>(sin inputs)</td></tr>"

        # Decisiones (ya vienen solo de nodos de decisión)
        dec_rows = ""
        for d in (self.decisiones or []):
            dec_rows += (
                "<tr>"
                f"<td style='border:1px solid #e2e8f0;padding:4px;'>{d.get('ts','')}</td>"
                f"<td style='border:1px solid #e2e8f0;padding:4px;'>{d.get('titulo','')}</td>"
                f"<td style='border:1px solid #e2e8f0;padding:4px;'>{d.get('seleccion','')}</td>"
                "</tr>"
            )
        if not dec_rows:
            dec_rows = "<tr><td colspan='3' style='border:1px solid #e2e8f0;padding:4px;'>(sin decisiones)</td></tr>"

        return widgets.VBox([
            widgets.HTML(f"""
            <div style="margin-top:12px;padding:12px;border-radius:12px;border:1px solid #e2e8f0;background:#ffffff;">
                <div style="font-size:12px;color:#0f172a;"><b>📌 INPUTS</b></div>
                <table style="width:100%;border-collapse:collapse;margin-top:8px;font-size:12px;">
                    <thead>
                        <tr>
                            <th style="border:1px solid #e2e8f0;padding:4px;text-align:left;">Campo</th>
                            <th style="border:1px solid #e2e8f0;padding:4px;text-align:left;">Valor</th>
                        </tr>
                    </thead>
                    <tbody>{inputs_rows}</tbody>
                </table>
            </div>
            """),
            widgets.HTML(f"""
            <div style="margin-top:12px;padding:12px;border-radius:12px;border:1px solid #e2e8f0;background:#ffffff;">
                <div style="font-size:12px;color:#0f172a;"><b>🧭 DECISIONES</b></div>
                <table style="width:100%;border-collapse:collapse;margin-top:8px;font-size:12px;">
                    <thead>
                        <tr>
                            <th style="border:1px solid #e2e8f0;padding:4px;text-align:left;">Timestamp</th>
                            <th style="border:1px solid #e2e8f0;padding:4px;text-align:left;">Nodo</th>
                            <th style="border:1px solid #e2e8f0;padding:4px;text-align:left;">Selección</th>
                        </tr>
                    </thead>
                    <tbody>{dec_rows}</tbody>
                </table>
            </div>
            """)
        ])

    def _render_footer(self):
        self.btn_volver.disabled = (len(self.historial) == 0)
        return widgets.VBox([
            widgets.HBox([self.btn_si, self.btn_no], layout={"justify_content":"space-between","margin":"10px 0"}),
            self.btn_volver,
            widgets.HTML("<div style='height:10px;'></div>"),
            self.btn_exportar,
            widgets.HTML("<div style='height:10px;'></div>"),
            self.block_panel,
            self.msg_box
        ])

    def _render(self):
        with self.output:
            self.output.clear_output()
            self._clear_msg()

            n = NODOS[self.nodo_id]
            header = self._render_header(n)

            if n["type"] == "task":
                body = self._render_task(n)
                self._decision_widget = None
            elif n["type"] == "decision":
                body, radios = self._render_decision(n)
                self._decision_widget = radios
                self._check_widgets = []
            elif n["type"] == "end":
                self._decision_widget = None
                self._check_widgets = []
                body = widgets.HTML(f"""
                <div style="margin-top:12px;padding:18px;border-radius:12px;border:2px solid #22c55e;background:#f0fdf4;">
                    <div style="font-size:20px;color:#0f172a;"><b>🏁 FIN</b></div>
                    <div style="margin-top:10px;font-size:15px;color:#0f172a;">{n.get('mensaje','')}
                    <div style="margin-top:10px;font-size:12px;color:#0f172a;">Estado final: <b>{n.get('estado_final','')}</b></div>
                </div>
                """)
            else:
                body = widgets.HTML("<div>Tipo de nodo no soportado.</div>")
                self._decision_widget = None
                self._check_widgets = []

            self._render_block_panel()
            end_tables = self._render_tables_if_end(n)
            footer = self._render_footer()
            self.main_box.children = [header, body, end_tables, footer]
            display(self.main_box)

    def _check_ready_to_advance(self):
        n = NODOS[self.nodo_id]

        if self.is_blocked:
            return False, "Paso bloqueado. Registre motivo(s) y use 'Rehacer paso'."

        if n["type"] == "end":
            return True, ""

        if n["type"] == "decision":
            if self._decision_widget is None or self._decision_widget.value is None:
                return False, "Debe seleccionar una opción para avanzar."
            return True, ""

        if n["type"] == "task":
            # Validación de inputs obligatorios
            if getattr(self, "_input_widgets", None):
                for w in self._input_widgets:
                    if getattr(w, "_pro_required", False) and not (w.value or "").strip():
                        return False, f"Debe completar el campo obligatorio: {getattr(w, "_pro_label", "")}"

            if self._check_widgets:
                required = [cb for cb in self._check_widgets if 'si aplica' not in (cb.description or '').lower()]
                if required and not all(cb.value for cb in required):
                    return False, "Debe completar el checklist antes de avanzar."
            return True, ""

        return True, ""

    def _advance_to(self, next_id):
        if next_id not in NODOS:
            self._set_msg(f"""
            <div style='margin-top:10px;padding:12px;border-radius:10px;background:#fff7ed;border:1px solid #fdba74;color:#0f172a;'>
                <b>⚠ Error de flujo:</b> el nodo destino no existe: <code>{next_id}</code>
            </div>
            """)
            self._log("ERROR_FLUJO", {"missing_next": next_id})
            return
        self.nodo_id = next_id
        self._render()

    def _on_si(self, _):
        ok, msg = self._check_ready_to_advance()
        if not ok:
            self._set_msg(f"""
            <div style='margin-top:10px;padding:12px;border-radius:10px;background:#fff1f2;border:1px solid #fecdd3;color:#0f172a;'>
                <b>⚠ {msg}</b>
            </div>
            """)
            self._log("VALIDACION_FALLA", {"mensaje": msg})
            return

        n = NODOS[self.nodo_id]

        if n["type"] == "end":
            self.estado = FINALIZADO
            self.end_ts = _now_iso()
            self._log("FINALIZA")
            self._set_msg("""
            <div style='margin-top:10px;padding:12px;border-radius:10px;background:#f1f5f9;border:1px solid #cbd5e1;color:#0f172a;'>
                <b>🏁 Ya está en el final/cierre.</b>
            </div>
            """)
            return

        self._push_hist()

        if n["type"] == "task":
            # Guardar inputs del paso (si existen)
            if getattr(self, "_input_widgets", None):
                for w in self._input_widgets:
                    key = getattr(w, "_pro_key", None)
                    if key:
                        self.inputs[key] = (w.value or "").strip()

            self._log("AVANZA", {"next": n.get("next")})
            self._advance_to(n.get("next"))
        elif n["type"] == "decision":
            chosen_next = self._decision_widget.value
            chosen_label = next((o["label"] for o in n.get("opciones",[]) if o["next"] == chosen_next), None)

            self.decisiones.append({
                "ts": _now_iso(),
                "nodo": self.nodo_id,
                "titulo": n.get("titulo",""),
                "seleccion": chosen_label,
                "next": chosen_next
            })
            self._log("DECISION", {"seleccion": chosen_label, "next": chosen_next})
            self._advance_to(chosen_next)

    def _on_no(self, _):
        if self.is_blocked:
            return
        self.is_blocked = True
        self.estado = BLOQUEADO
        self.block_ts_inicio = _now_iso()
        self._log("BLOQUEADO_INICIO")
        self._render()

    def _on_rehacer(self, _):
        motivos = list(self.sel_motivos.value) if hasattr(self, "sel_motivos") else []
        detalle = (self.txt_detalle.value or "").strip() if hasattr(self, "txt_detalle") else ""

        if not motivos:
            self._set_msg("""
            <div style='margin-top:10px;padding:12px;border-radius:10px;background:#fff1f2;border:1px solid #fecdd3;color:#0f172a;'>
                <b>⚠ Debe seleccionar al menos un motivo.</b>
            </div>
            """)
            self._log("BLOQUEADO_VALIDACION_FALLA", {"mensaje": "sin_motivo"})
            return

        if "Otro" in motivos and not detalle:
            self._set_msg("""
            <div style='margin-top:10px;padding:12px;border-radius:10px;background:#fff1f2;border:1px solid #fecdd3;color:#0f172a;'>
                <b>⚠ Debe ingresar detalle si selecciona 'Otro'.</b>
            </div>
            """)
            self._log("BLOQUEADO_VALIDACION_FALLA", {"mensaje": "otro_sin_detalle"})
            return

        bloqueo = {
            "ts_inicio": getattr(self, "block_ts_inicio", None),
            "ts_fin": _now_iso(),
            "nodo": self.nodo_id,
            "titulo": NODOS[self.nodo_id].get("titulo",""),
            "motivos": motivos,
            "detalle": detalle
        }
        self.bloqueos.append(bloqueo)
        self._log("BLOQUEADO_FIN", bloqueo)

        self.is_blocked = False
        self.estado = EN_CURSO
        self._log("REHACER_PASO")
        self._render()

    def _on_volver(self, _):
        if self.is_blocked:
            self._set_msg("""
            <div style='margin-top:10px;padding:12px;border-radius:10px;background:#fff1f2;border:1px solid #fecdd3;color:#0f172a;'>
                <b>⚠ No puede volver mientras el paso está bloqueado. Use 'Rehacer paso'.</b>
            </div>
            """)
            return

        prev_id = self._pop_hist()
        if prev_id is not None:
            self._log("VOLVER", {"to": prev_id})
            self._advance_to(prev_id)

    def _on_exportar(self, _):
        payload = {
            "proceso": "PRO141 – Tratamiento de materiales obsoletos y análisis de obsolescencia",
            "run_id": self.run_id,
            "estado": self.estado,
            "start_ts": self.start_ts,
            "end_ts": self.end_ts,
            "current_node": self.nodo_id,
            "history_stack": list(self.historial),
            "decisiones": list(self.decisiones),
            "bloqueos": list(self.bloqueos),
            "inputs": dict(self.inputs),
            "logs": list(self.logs),
            "export_ts": _now_iso()
        }
        pretty = json.dumps(payload, ensure_ascii=False, indent=2)
        self._set_msg(f"""
        <div style='margin-top:10px;padding:12px;border-radius:10px;background:#f1f5f9;border:1px solid #cbd5e1;color:#0f172a;'>
            <b>📦 Export JSON (trazabilidad)</b>
            <pre style='white-space:pre-wrap;margin-top:10px;color:#0f172a;'>{pretty}</pre>
        </div>
        """)

    def iniciar(self):
        display(self.output)

hmi = PRO141HMI()
hmi.iniciar()


Output(layout=Layout(width='100%'))